In [2]:
import pandas as pd
import xlrd
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import datetime
import math

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [28]:
folder_podataka = "./PISpodaci"
pregled = pd.read_excel("../samoPouzdaniPodaciTabela.xlsx", converters= {'kraj': pd.to_datetime, 'pocetak': pd.to_datetime})
preslikavanje_opstina = {"Novi Sad": "NS", "Ruma":"RU","Sombor":"SO"}
preslikavanje_lokacija = {"Cenej": "CE" ,"Novi Slankamen":"NS","Ridjica":"RI","Sremski Karlovci":"SK"}
z=None
#za svaku PIS tabelu, cemo napraviti korespondirajucu PIS tabelu, sa pouzdanim podacima + vrsta i mesto
# VAZNO !! Moraju biti prebaceni u xls format posle skidanja sa PIS sajta. To se moze uraditi otvaranjem u Excelu i cuvsanju u xls formatu
for fname in os.listdir(folder_podataka):
    print (fname)
    #IZVLACENJE AMS naziva iz imena fajla !
    ams =fname.split('_')[0] ## REFACTOR - ovo mora da bude regex. 
    #assert(len(ams)==8)
    print(ams)
    assert(re.match("0+[0-9A-F]+",ams)) # REFACTOR - napravi bolji regex, more robust
    #IZVLACENJE PODATAKA IZ PREGLED TABELE
    record_for_ams = pregled[pregled.ams==ams] #za ovu ams samo trzimo records
    record_for_ams = record_for_ams.fillna(pd.Timestamp.today())#najbolje bi ih bilo napuniti sa poslednjom vrednoscu tabele REFACTOR Ovo znaci da ako nije oznaceno, onda znaci da je jos uvek tamo ovaj, tako da mozemo sve podatke uzeti
    #UCITAVANJE TABELE
    PIStabela=pd.read_excel(os.path.join(folder_podataka,fname),header=[0,1])
    PIStabela = PIStabela.sort_index()
    #flattening the header
    #SREDJIVANJE TABELE!!
    PIStabela.columns = [' '.join(col).strip() for col in PIStabela.columns.values]
    columns_to_drop = []
    for col in PIStabela.columns :
        if ("water" in col.lower()) or ("battery" in col.lower()):
            columns_to_drop.append(col)
            #    if (any("water" in str_.lower() for str_ in col) or any("battery" in str_.lower() for str_ in col)):
    print(columns_to_drop)        
    PIStabela=PIStabela.drop(columns=columns_to_drop)
#z = pd.concat(z.values(),axis=0)
    assert(len(PIStabela.columns)==13)
    #prvo cemo ih iseci, i onda cemo ih spojiti u jednu tabelu !!
    tabele_za_jedan_ams = []
    #idemo kroz PREGLED tabelu i za svaku stavku odvojim deo PIS tabele, koja je validna BTW
    for index,row in record_for_ams.iterrows(): #dobro je ovo, za ea5 se trazi, to nam treba
        assert(row.pocetak< row.kraj)
        print (row.pocetak,row.kraj)
        izuzet_dan_postavljanja = (row.pocetak+pd.offsets.DateOffset(days=1))
        
        #A dan skidanja?????????
        novi = PIStabela.loc[izuzet_dan_postavljanja:row.kraj].copy() #pravimo novi frejm koji je kopija tog perioda, krecemo posle dana postavljanja, posto za to znamo da su nepouzdani TODO mozda napraviti malo preciznije odsecanje 
        novi["opstina"] = row.opstina #popunjavamo podatke, koje sad znamo za sigurno
        novi["mesto"] = row.mesto
        novi["usev"] = row.usev
        tabele_za_jedan_ams.append(novi) #jos uvek smo kod istog AMS-a, i sve sto nadjemo cemo nakaciti
        #print (novi[pd.isna(novi.iloc[:,0:3][novi.iloc[:,0:3].applymap(np.isreal)])])
        print (novi.iloc[:,0:3][novi.iloc[:,0:3].isnull().any(axis=1)])
        print (len(novi.iloc[:,0:3]))
        novi.to_csv("pouzdani_%s_%s_%s_%s_%s.csv"%(ams,preslikavanje_opstina[row.opstina],preslikavanje_lokacija[row.mesto],row.usev,row.pocetak))    
        frejmovi_za_ams = pd.concat(tabele_za_jedan_ams)
        frejmovi_za_ams.to_csv("%s_pouzdaniPeriodi_biljneVrste_lokacije.csv" %(ams))
        idx_ref = pd.DatetimeIndex(start=novi.index[0], end=novi.index[-1], freq="h")
        gaps = idx_ref[~idx_ref.isin(novi.index)]
        print (gaps)
        df = pd.DataFrame(index=gaps)
        df["opstina"]= row.opstina
        df["mesto"] = row.mesto
        df["usev"] = row.usev
        
        df.to_csv("missing_periods_%s" %(ams+row.opstina+row.mesto +row.usev))

00000EA5_station_data.xlsx
00000EA5
['Water mark [cbar] avg', 'Water mark [cbar] avg.1', 'Water mark [cbar] avg.2', 'Water mark [cbar] avg.3', 'Water mark [cbar] avg.4', 'Water mark [cbar] avg.5', 'Battery voltage [mV] last']
2010-04-20 00:00:00 2013-03-29 00:00:00
Empty DataFrame
Columns: [HC Air temperature [°C] avg, HC Air temperature [°C] max, HC Air temperature [°C] min]
Index: []
25746
DatetimeIndex(['2010-10-31 00:00:00', '2011-10-30 00:00:00',
               '2012-10-20 17:00:00', '2012-10-20 18:00:00',
               '2012-10-20 19:00:00', '2012-10-20 20:00:00',
               '2012-10-28 00:00:00'],
              dtype='datetime64[ns]', freq=None)
2013-03-29 00:00:00 2019-01-26 19:14:06.493811
                     HC Air temperature [°C] avg  HC Air temperature [°C] max  \
2016-04-23 14:00:00                          NaN                          NaN   
2016-04-24 12:00:00                          NaN                          NaN   
2016-04-30 02:00:00                         

DatetimeIndex(['2011-05-05 08:00:00', '2011-05-05 09:00:00',
               '2011-05-05 10:00:00', '2011-05-05 11:00:00',
               '2011-10-30 00:00:00', '2012-10-20 17:00:00',
               '2012-10-20 18:00:00', '2012-10-20 19:00:00',
               '2012-10-20 20:00:00', '2012-10-20 21:00:00',
               ...
               '2017-05-12 02:00:00', '2017-05-12 03:00:00',
               '2017-05-12 04:00:00', '2017-05-12 05:00:00',
               '2017-05-12 06:00:00', '2017-05-12 07:00:00',
               '2017-05-12 08:00:00', '2017-05-12 09:00:00',
               '2017-05-12 10:00:00', '2017-05-12 11:00:00'],
              dtype='datetime64[ns]', length=352, freq=None)
00000E9A_station_data.xls
00000E9A
['Watermark  [cbar] avg', 'Watermark [cbar] avg', 'Watermark [cbar] avg.1', 'Watermark  [cbar] avg.1', 'Watermark [cbar] avg.2', 'Watermark [cbar] avg.3', 'Battery Voltage  [mV] last']
2013-08-02 00:00:00 2018-11-19 00:00:00
                     HC Air Temperature  [°C] avg

In [112]:
zaracun = pd.read_csv("NoviSadCenejZaRacun.csv")
#zaracun["HC Air Temperature [°C] avg"]
#zaracun = zaracun.set_index(zaracun.Date)
zbir = 0
end_of_chilling_index = None
end_of_chilling_date = None
chilling_units_max = 1800
for index, row in zaracun.iterrows(): 
    if zbir>=chilling_units_max:# and index>=137*24: #varirati ovaj broj chilling unitsa 
        print ("broj cvetanja je %s" %index)
        end_of_chilling_index = index
        end_of_chilling_date = row.Date
        print (row)
        print (zbir)
        break
    if row.Tavg <= 1.4:
        # Do the thing
        zbir+=0
    elif row.Tavg <=2.4:
        zbir+=0.5
            # Do the other thing
    if row.Tavg <= 9.1:
        zbir+=1
                # Fall-through by not using elif, but now the default case includes case 'a'!
    elif row.Tavg <=12.4:
        zbir+=0.5
                    # Do yet another thing
    elif row.Tavg <=15.9:
        zbir+=0

    elif row.Tavg <=18.0:
        zbir+=-0.5
    else:
        zbir+=-1
        
        
   
    
Tu = 25 # optimum temperature
Tc = 36 # critical temperature
Tb = 4 #base temperature
F = 1 # plant stress factor, 1 where no particular stress
GDH = 0
for i, row in zaracun.iloc[end_of_chilling_index:].iterrows():
#for i, row in zaracun.iterrows():    #print (GDH)
    if GDH>=6100:
        print ("GDH je %s" %GDH)
        print (row)
        break
    if (row.Tavg >=Tb) and (row.Tavg < Tu):
        z =  F*((Tu-Tb)/2)*(1+ math.cos(math.pi + math.pi*((row.Tavg -Tb)/(Tu-Tb))))
        GDH+=z  
    elif (row.Tavg >=Tu) and (row.Tavg<=Tc):
        z= F*(Tu-Tb)*(1+ math.cos(math.pi/2 + (math.pi/2)*((row.Tavg -Tu)/(Tc-Tu))))
        GDH+=z
    else:
        GDH+=0
    
        

                                # Do the default
    

    
    
#T <= 1.4    = 0
#1.4  < T <= 2.4    = 0.5
#2.4  < T <= 9.1    = 1
#9.1  < T <= 12.4   = 0.5
#12.4  < T <= 15.9    = 0
#15.9 < T <= 18.0    = -0.5
#T > 18.0    = -1


broj cvetanja je 1993
Date    23/01/2016 01:00
Tavg               -8.02
Tmax               -7.48
Tmin               -8.94
Name: 1993, dtype: object
1800.5
GDH je 6111.6371697463355
Date    08/04/2016 02:00
Tavg               16.73
Tmax               16.98
Tmin               16.37
Name: 3818, dtype: object


In [104]:
zbir2=0

for index, row in zaracun.iterrows(): 
    if zbir2>=50 and index>=250*24:
        print ("broj cvetanja je %s" %index)
        end_of_chilling_index = index
        end_of_chilling_date = row.Date
        print (row)
        print (zbir2)
        break
    elif row.Tavg >0 and row.Tavg <7.2:
        zbir2+=1
        
    else:
        zbir2+=0
        

broj cvetanja je 6000
Date    08/04/2017 00:00
Tavg                7.72
Tmax                7.92
Tmin                7.58
Name: 6000, dtype: object
1502


In [114]:
pd.read_csv?

In [67]:
#pregled.set_index(['ams','opstina','mesto'])
#prvo pretrazi iz ams kolone sve koji spadaju
#ovo takodje treba u petlju, tako da cemo preko svih preci, bilo bi naravno dobro citati iz tabele, tj. mozemo da napravimo unikatne i onda po njima citamo fajlove, znaci for ams in ams.unique() i onda iscitamo fajlove redom
#record_for_ams = pregled[pregled.ams==ams]
#import datetime

# onda iterira kroz njih i gleda dobre periode
# kaze : stigli smo do prvog 00000ea5, izvadimo valjani period, trazimo ga u tabeli, ako ga nadjemo onda
# popunimo tabelu sa opstinom, mestom i usevom, i tako sve dok nema vise sta iz te tabele sa PIS-a da se izvuce
# zapisuju se periodi koji su se izbacili
# cuva se nova (cista) datoteka sa svime
#record_for_ams = record_for_ams.fillna(pd.Timestamp.today())#najbolje bi ih bilo napuniti sa poslednjom vrednoscu tabele REFACTOR
#z = z.sort_index()
tabele_za_jedan_ams = []

for index,row in record_for_ams.iterrows(): #dobro je ovo, za ea5 se trazi, to nam treba
    assert(row.pocetak< row.kraj)
    print (row.pocetak,row.kraj)
    novi = z.loc[row.pocetak:row.kraj].copy()
    novi["opstina"] = row.opstina
    novi["mesto"] = row.mesto
    novi["usev"] = row.usev
    tabele_za_jedan_ams.append(novi)
    print (novi)
    novi.to_csv("pouzdani_%s_%s_%s_%s_%s.csv"%(ams,row.opstina,row.mesto,row.usev,row.pocetak))    
frejmovi_za_ams = pd.concat(tabele_za_jedan_ams)
frejmovi_za_ams.to_csv("%s_pouzdaniPeriodi.csv" %(ams))
#frejmovi_za_ams.to_csv("%s_pouzdaniPeriodi_%s_%s.csv" %(ams,"".join([vrsta for vrsta in frejmovi_za_ams["usev"].unique()]), "".join([mesto for mesto in frejmovi_za_ams["mesto"].unique()])))
    
#TODO - ja bih za jednu ea5 u jednu tabelu
    
#Sad treba da ih spojimo, popunimo kolone sa nazivom mesta i sortom, i mozemo ih sacuvati pod EA5_pouzdani_periodi_vrste_lokacije
    
    
##IMPORTANT On ih odsece do granice,ali uzme i 00:00:00 od granice. Znaci treba mozda da se promeni kood, kako ne bismo uzimali taj prvi u ponoc, doduse on nam nije sporan obicno posto nisu u ponoc sklanjali, ali nekako strci

#Treba razmisiti o tome koji ce se podaci ukljuciti u tabelu. da li postoji neki podaci kojima ne znamo vrstu. Mislim da ne postoje,tj. mislim da je sve zapisano u tabelu, ovo ostalo mozemo izopstiti. Pazi ako za nesto znamo mesto, ne znamo vrstu, i to je ok, to nam je korisna informacija
    


2010-04-21 00:00:00 2013-03-29 00:00:00
                     HC Air temperature [°C] avg  HC Air temperature [°C] max  \
2010-04-21 00:00:00                         9.95                        10.45   
2010-04-21 01:00:00                         9.00                         9.48   
2010-04-21 02:00:00                        10.08                        11.11   
2010-04-21 03:00:00                         9.79                        10.10   
2010-04-21 04:00:00                         9.29                        10.04   
2010-04-21 05:00:00                         9.28                         9.57   
2010-04-21 06:00:00                         9.61                        10.34   
2010-04-21 07:00:00                        10.29                        11.19   
2010-04-21 08:00:00                        12.88                        14.21   
2010-04-21 09:00:00                        14.61                        15.38   
2010-04-21 10:00:00                        15.68                     

2013-03-30 00:00:00 2018-12-19 19:33:46.207790
                     HC Air temperature [°C] avg  HC Air temperature [°C] max  \
2013-03-30 00:00:00                         4.39                         4.54   
2013-03-30 01:00:00                         4.67                         4.89   
2013-03-30 02:00:00                         4.69                         4.85   
2013-03-30 03:00:00                         4.80                         4.92   
2013-03-30 04:00:00                         5.11                         5.27   
2013-03-30 05:00:00                         4.87                         5.05   
2013-03-30 06:00:00                         4.64                         4.72   
2013-03-30 07:00:00                         4.95                         5.34   
2013-03-30 08:00:00                         6.09                         6.29   
2013-03-30 09:00:00                         6.80                         7.01   
2013-03-30 10:00:00                         7.95              

In [ ]:
#ovo mozda kasnije, ne verujem da cemo koristiti posto ne radimo sa tim podacima, vec sa podacima sa PIS-a

z=pd.read_excel("Nestin.xlsx",usecols="A:D,M:O", names=["Date","soil_t_avg","soil_t_min","soil_t_max","air_t_avg","air_t_min","air_t_max"],header=1)

z.set_index(keys="Date",inplace=True) #i ovo bi sve trebalo u ucitavanju
z.index = pd.to_datetime(z.index)

for col in z:
    z[col] = pd.to_numeric(z[col]) #ovo moze vec u ucitavanju, samo ne smemo za datetime 



In [29]:
pregled = pd.read_excel("../samoPouzdaniPodaciTabela.xlsx", converters= {'kraj': pd.to_datetime, 'pocetak': pd.to_datetime})
pregled



,ams,opstina,mesto,pocetak,kraj,usev,komentar
0,00000EA5,Ruma,Novi Slankamen,2010-04-21,2013-03-29,Jabuka Ogled,premestanje stanice 29.03.2013 od 12:00 - 14:0...
1,00000EA5,Ruma,Novi Slankamen,2013-03-30,NaT,Jabuka Ogled,NaN
2,00000E94,Sombor,Ridjica,2010-04-17,NaT,Jabuka,NaN
3,00000E88,Novi Sad,Sremski Karlovci,2010-04-27,NaT,VInova Loza,NaN


In [28]:
for col in z.columns :
    print (col)
    if ("water" in col.lower()) or ("battery" in col.lower()):
#    if (any("water" in str_.lower() for str_ in col) or any("battery" in str_.lower() for str_ in col)):
        z=z.drop(columns=col)
#z = pd.concat(z.values(),axis=0)
assert(len(z.columns)==13)


HC Air temperature [°C] avg
HC Air temperature [°C] max
HC Air temperature [°C] min
Dew Point [°C] avg
Dew Point [°C] min
HC Relative humidity [%] avg
HC Relative humidity [%] max
HC Relative humidity [%] min
Precipitation [mm] sum
Leaf Wetness [min] time
Water mark [cbar] avg
Water mark [cbar] avg.1
Water mark [cbar] avg.2
Water mark [cbar] avg.3
Water mark [cbar] avg.4
Water mark [cbar] avg.5
Soil temperature [°C] avg
Soil temperature [°C] max
Soil temperature [°C] min
Battery voltage [mV] last
13


In [27]:

z=pd.read_excel("../%s" %fname,header=[0,1])
#flattening the header
z.columns = [' '.join(col).strip() for col in z.columns.values]





In [ ]:
#NEke misli : 
# Mislim bi trebalo posle ovoga da imam pouzdane tabele, sve sa ovim svim informacijama da. I onda cu ih sve spojiti i tu ce mi ##Multindex come in handy posto cu moci da ih grupisem po masini po mestu, i taman cu videti gde seta masina, svakako ams nije index, ali bice multindex kad se spoji sa mestom i datumom namestanja
#Ono sto pise novi slankamen na sajtu, ne znaci da je novi slankamen, treba se pratiti sve u tabeli prebacivanja

#Sledeci koraci : Napraviti tabelu prebacivanja, bar neki kostur i napisati kod koji ide po toj tabeli 

#E94 je


hdf.loc[pd.IndexSlice[:, 'ORD', 'DSM', :, '2014-01-01':'2014-01-15'],
        ['dep_time', 'dep_delay', 'arr_time', 'arr_delay']]

#ovo gore je super stvarno, znaci mozemo stagod hocemo, i datume odrediti kako zelimo da gledamo
# super je zato sto sad mozemo da odvojimo te datume koje su nam okej, znaci test['pocetak']:test['kraj']

#znaci nesto ovako

svipodacizajedno.loc[pd.IndexSlice['e8c','novi sad','ruma',pregledTabela['pocetak']]]

#ovaj pipe je svakako dobar. moze to_datetime da se izvrsi. , moze rename, moze 
df = (pd.read_csv(fp)
            .rename(columns=str.lower)
            .drop('unnamed: 36', axis=1)
            .pipe(extract_city_name)
            .pipe(time_to_datetime, ['dep_time', 'arr_time', 'crs_arr_time', 'crs_dep_time'])
            .assign(fl_date=lambda x: pd.to_datetime(x['fl_date']),
                    dest=lambda x: pd.Categorical(x['dest']),
                    origin=lambda x: pd.Categorical(x['origin']),
                    tail_num=lambda x: pd.Categorical(x['tail_num']),
                    unique_carrier=lambda x: pd.Categorical(x['unique_carrier']),
                    cancellation_code=lambda x: pd.Categorical(x['cancellation_code'])))

#za date mozemo assign to_datetime. Cisto da sve bude kul i ok 

In [3]:

#Nisam sigurna da sam dobro prekucala 
#ovo ide kasnije, kad imamo ciste podatke
import os
from unidecode import unidecode
import pandas as pd
#Ova skripta se pokrece iz METEO PODACI SVE foldera

for subdir,dirs,files in os.walk('.'):
    opstina = os.path.split(subdir)[-1]
    gen = (x for x in files if "~" not in x)
    for x in gen:
        opstina = unidecode(opstina.decode("utf-8")).lower().title()
        tabl = pd.read_excel(os.path.join(subdir,x))
        lokacija = os.path.splitext(x)[0] #ovde mora prvo ici check iz one tabele,znaci cekiracemo u tabeli i samo izvuci pouzdane podatke sa pouzdanim lokacijama 
        lokacija = unidecode(lokacija.decode("utf-8")).lower().title()
        tabl['opstina'] = opstina
        tabl['lokacija'] = lokacija
        tabl.to_csv(opstina+"-" + lokacija + ".csv", encoding = "utf-8")

ModuleNotFoundError: No module named 'unidecode'

In [26]:
#z=pd.read_csv("../00000EA5_station_data.csv",usecols="1:4,M:O", names=["Date","soil_t_avg","soil_t_min","soil_t_max","air_t_avg","air_t_min","air_t_max"],header=1)
fname = "00000EA5_station_data.xlsx" #TODO convertovati automatski csv u xlsx
#regex to extract station
ams =fname.split('_')[0] ## REFACTOR - ovo mora da bude regex. 
#assert(len(ams)==8)
print(ams)

m = re.match("0+[0-9A-F]+",ams) # REFACTOR - napravi bolji
assert (m)



00000EA5
